In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

### 3 -  Loading Different Document Formats



In [2]:
!pip install -r requirements.txt -q

In [3]:
!pip3 install --upgrade pip

In [4]:
#!pip install chromadb -q

In [5]:
# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data

In [6]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter  
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0) 
    chunks = text_splitter.split_documents(data) 
    return chunks

In [7]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')
    
    
# print_embedding_cost(chunks)

In [8]:
# create embeddings using OpenAIEmbeddings() and save them in a Chroma vector store
def create_embeddings(chunks):
    from langchain.vectorstores import Chroma
    from langchain.embeddings.openai import OpenAIEmbeddings
    embeddings = OpenAIEmbeddings()
    vector_store = Chroma.from_documents(chunks, embeddings)
    return vector_store

In [9]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.2)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    answer = chain.run(q)
    return answer


### RUNNING CODE

In [10]:
data = load_document('files/state_of_the_union.txt')
print (f'You have {len(data)} pages in your data')

RuntimeError: Error loading files/state_of_the_union.txt

In [ ]:
chunks = chunk_data(data)
print(len(chunks))

In [ ]:
vector_store = create_embeddings(chunks)

In [ ]:
q = 'what is the whole document about?'
# q = 'what is resoning and acting in LLMs?'
# q = 'Summarize the entire document in a few paragraphs.'

k = 3
answer = ask_and_get_answer(vector_store, q, k)
print(answer)

In [ ]:
import time

i = 1
print('Write Quit of Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i+1
    if q.lower() in  ['quit', 'exit']:
        print('Quitting ... Bye Bye!')
        time.sleep(2)
        break
        
    answer = ask_and_get_answer(vector_store, q, 5)
    print(answer)
    print(f'\n {"-"*50} \n')
    

In [ ]:
def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(temperature=0.1)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":5})
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    
    return result, chat_history

In [ ]:
chat_history = list()

q = 'What did the president say about Ketanji Brown Jackson?'
result, chat_history = ask_with_memory(vector_store, q, chat_history)

print(result['answer'])
print('-' * 100)
print(chat_history)  # for debugging

In [ ]:
q = 'Did he mention who she succeeded?'
result, chat_history = ask_with_memory(vector_store, q, chat_history)

print(result['answer'])
print('-' * 100)
print(chat_history)

### Ask with Memory Loop

In [ ]:
import time
i = 1

chat_history = []

print("Write Quit or Exit to quit")
while True:
    q = input(f"Question #{i}")
    i = i + 1
    if q.lower() in ["quit","exit"]:
        print("Qutting")
        time.sleep(2)
        break
    result, _ = ask_with_memory(vector_store, q, chat_history)
    print (result['answer'])
    print("----------------------------------------------------------------------")